In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report


In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to the new dataset and saved model
new_dataset_dir = "../Test"  # Replace with the path to your dataset
model_load_path = "flame3_resnet50.pth"  # Path to the saved model

# Define transforms (same as used for training)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match input size for ResNet
    transforms.ToTensor(),
])

# Load dataset
test_dataset = datasets.ImageFolder(root=new_dataset_dir, transform=test_transform)

# Create DataLoader
batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [3]:
# Load the model
from torchvision import models

# Initialize the ResNet50 model
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # Binary classification (Fire vs No_Fire)
model.load_state_dict(torch.load(model_load_path))
model = model.to(device)
model.eval()  # Set model to evaluation mode



/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/local_scratch/slurm.1342496/ipykernel_766225/3702014094.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
# Function to evaluate the model
def evaluate_model(model, dataloader):
    all_preds = []
    all_labels = []

    with torch.no_grad():  # No gradient computation for testing
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    report = classification_report(
        all_labels, 
        all_preds, 
        target_names=test_dataset.classes, 
        output_dict=True
    )
    
    # Extract metrics
    accuracy = report["accuracy"] * 100
    f1_score = report["weighted avg"]["f1-score"]
    precision = report["weighted avg"]["precision"]
    recall = report["weighted avg"]["recall"]

    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"F1 Score: {f1_score:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")

# Evaluate on the test dataset
evaluate_model(model, test_loader)


Test Accuracy: 57.33%
F1 Score: 0.51
Precision: 0.52
Recall: 0.57


In [6]:
import os

# Paths to the directories
fire_dir = "../Test/Fire"  # Replace with the path to the 'Fire' directory
nofire_dir = "../Test/No_Fire"  # Replace with the path to the 'No_Fire' directory

# Function to count images in a directory
def count_images(directory):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')  # Add other extensions if needed
    return sum(1 for file in os.listdir(directory) if file.lower().endswith(image_extensions))

# Count images in both directories
num_fire_images = count_images(fire_dir)
num_nofire_images = count_images(nofire_dir)

# Print the results
print(f"Number of images in 'Fire' directory: {num_fire_images}")
print(f"Number of images in 'No_Fire' directory: {num_nofire_images}")
# print(f"Total number of images: {num_fire_images + num_nofire_images}")


Number of images in 'Fire' directory: 5137
Number of images in 'No_Fire' directory: 3480
